In [ ]:
import os
import numpy as np
from scipy.linalg import sqrtm
from skimage.metrics import structural_similarity as ssim
from skimage.metrics import peak_signal_noise_ratio as psnr
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

def load_images_from_folder(folder_path):
    images = []
    for filename in os.listdir(folder_path):
        if filename.lower().endswith(('.png', '.jpg', '.jpeg', '.tiff', '.bmp', '.gif')):
            img_path = os.path.join(folder_path, filename)
            try:
                img = np.array(Image.open(img_path).convert('RGB'))
                images.append(img)
            except Exception as e:
                print(f"Error loading image {filename}: {e}")
    return images

def calculate_fid(real_images, synthetic_images):
    inception = models.inception_v3(pretrained=True)
    inception.eval()
    preprocess = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])

    def get_features(images):
        features = []
        for img in images:
            img_t = preprocess(Image.fromarray(img)).unsqueeze(0)
            with torch.no_grad():
                feat = inception(img_t)[0].squeeze().cpu().numpy()
            features.append(feat)
        return np.array(features)

    real_features = get_features(real_images)
    synthetic_features = get_features(synthetic_images)

    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = synthetic_features.mean(axis=0), np.cov(synthetic_features, rowvar=False)

    ssdiff = np.sum((mu1 - mu2)**2.0)
    
    # Handle potential numerical instability
    eps = 1e-6
    sigma1 = sigma1 + np.eye(sigma1.shape[0]) * eps
    sigma2 = sigma2 + np.eye(sigma2.shape[0]) * eps

    try:
        covmean = sqrtm(sigma1.dot(sigma2))
        if np.iscomplexobj(covmean):
            covmean = covmean.real
        fid = ssdiff + np.trace(sigma1 + sigma2 - 2.0 * covmean)
    except ValueError:
        print("Warning: FID calculation failed. Returning infinity.")
        fid = float('inf')

    return fid

def calculate_ssim(real_image, synthetic_image):
    return ssim(real_image, synthetic_image, multichannel=True)

def calculate_psnr(real_image, synthetic_image):
    return psnr(real_image, synthetic_image)

def calculate_mse(real_image, synthetic_image):
    return np.mean((real_image.astype(float) - synthetic_image.astype(float)) ** 2)

def calculate_autocorrelation(image):
    gray = np.mean(image, axis=2)
    result = np.correlate(gray.flatten(), gray.flatten(), mode='full')
    return result[result.size // 2:] / np.max(result)

def evaluate_images(real_images_folder, synthetic_images_folder):
    real_images = load_images_from_folder(real_images_folder)
    synthetic_images = load_images_from_folder(synthetic_images_folder)

    if len(real_images) == 0 or len(synthetic_images) == 0:
        raise ValueError("No images found in one or both folders.")

    if len(real_images) != len(synthetic_images):
        print("Warning: Number of real and synthetic images do not match.")
        min_images = min(len(real_images), len(synthetic_images))
        real_images = real_images[:min_images]
        synthetic_images = synthetic_images[:min_images]

    fid = calculate_fid(real_images, synthetic_images)
    
    ssim_scores = []
    psnr_scores = []
    mse_scores = []
    autocorr_scores = []

    for real, synth in zip(real_images, synthetic_images):
        ssim_scores.append(calculate_ssim(real, synth))
        psnr_scores.append(calculate_psnr(real, synth))
        mse_scores.append(calculate_mse(real, synth))
        autocorr_scores.append(np.mean(calculate_autocorrelation(synth)))

    return {
        "FID": fid,
        "SSIM": np.mean(ssim_scores),
        "PSNR": np.mean(psnr_scores),
        "MSE": np.mean(mse_scores),
        "Autocorrelation": np.mean(autocorr_scores)
    }

# Example usage
real_images_folder = "original"
synthetic_images_folder = "synthetic"

try:
    results = evaluate_images(real_images_folder, synthetic_images_folder)
    for metric, value in results.items():
        print(f"{metric}: {value}")
except Exception as e:
    print(f"An error occurred: {e}")